In [ ]:
#에러 코드 핸들링
import os
import warnings

import numpy as np
import matplotlib.pyplot as plt

# 이미지 -> 디렉토리 및 라벨링
from keras.preprocessing.image import ImageDataGenerator
#from keras.utils import to_categorical

#모델링
from keras.models import Sequential,load_model 
from tensorflow.keras.layers import  Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout,experimental

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings("ignore", category=DeprecationWarning) 


# 모델 입력
My_Model = input()
#My_Model = 'model-201814132'

# 데이터 증폭
'''
validation_split = 0.2,  => train : validation 8:2 
rotation_range=10,      =>회전 각도 10도까지
zoom_range = 0.3,       => 확대 범위
width_shift_range=0.2,  => 좌우 이동 범위
height_shift_range=0.2) => 상하 이동 범위
'''
train_data_augmentation = ImageDataGenerator(
    validation_split = 0.2,
    rotation_range=10,
    zoom_range = 0.3,
    width_shift_range=0.2,
    height_shift_range=0.2)

#train 데이터 설정 
train_data = train_data_augmentation.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=150,
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        class_mode='categorical')


# validation 데이터 설정
validation_data = train_data_augmentation.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=100,
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        class_mode='categorical',
        subset='validation')

#test 데이터 설정
test_data = ImageDataGenerator().flow_from_directory(
        './images2/test',
        classes = ['food','interior', 'exterior'],
        target_size=(300,300),
        batch_size=100,

        class_mode='categorical')

model = Sequential([
    
Input(shape=(300,300,3), name='input_layer'),
experimental.preprocessing.Rescaling(1./255),
    
Conv2D(32,(3,3), activation='relu',name='Conv_layer1'),
MaxPooling2D(pool_size=(2,2),name='Pooling_layer2'),
    
Conv2D(96,(3,3), activation='relu',name='Conv_layer3'),
MaxPooling2D(pool_size=(2,2),name='Pooling_layer4'),

Conv2D(64,(3,3), activation='relu',name='Conv_layer5'),
MaxPooling2D(pool_size=(2,2),name='Pooling_layer6'),
    

Dropout(0.5),
Conv2D(128,(3,3), activation='relu',name='Conv_layer7'),
MaxPooling2D(pool_size=(2,2),name='Pooling_layer8'),

Flatten(),
Dense(160, activation='relu',name='FC_layer9', kernel_initializer='glorot_uniform',),
Dense(20, activation='relu',name='FC_layer10', kernel_initializer='glorot_uniform',),
    
Dense(3, activation='softmax', name='output_layer11')
])

 

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



a = datetime.datetime.now()
print('fitting start', a)

# 4. 모델 fitting
def model_():
    history = model.fit(

        train_data,

        steps_per_epoch=15,

        epochs=30,

        validation_data=validation_data,

        validation_steps=10)

    model.save('model-201814132')
    return history


model.save('model-201814132')
b = datetime.datetime.now()
#history = model_()
print('Fitting End', b)
print('Fitting Time =' , b-a)


def plot_loss_curve(history):
    
    plt.figure(figsize = (5,3))
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])

    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['train','test'], loc = 'upper right')
    plt.show

    
def plot_accuracy_curve(history):

    plt.figure(figsize = (5,3))
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])

    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['train','test'], loc = 'upper right')
    plt.show

plot_accuracy_curve(history.history)

print('train accuracy ={} , validation accuracy ={}' .format(
                                                            round(history.history['accuracy'][-1],5), 
                                                            round(history.history['val_accuracy'][-1],5) ))
plot_loss_curve(history.history)
print('train loss ={} ,     validation loss ={}' .format(
                                                        round(history.history['loss'][-1],5), 
                                                        round(history.history['val_loss'][-1],5) ))


X,y = test_data.next()

def print_class(Num):
    class_of_Image = '음식' if Num == 0 else '실내' if Num==1 else '실외'
    return class_of_Image

def predict_image_sample_Noprint(model,X_test, y_test,test_id= -1 ):

    if test_id <0 :
        from random import randrange
        test_sample_id = randrange(len(y))
    else:
        test_sample_id = test_id
        
    test_image =X_test[test_sample_id]

    test_image = test_image.reshape(1,300,300,3)
    

    y_actual = y_test[test_sample_id]

    y_actual_num = np.argmax(y_actual)
    actual_class = print_class(y_actual_num)


    y_pred = model.predict(test_image)
    
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    predict_class = print_class(y_pred_num)

    if y_pred_num != y_actual_num:
        with open("wrong_samples.txt", "a") as errfile:
            print('%d'%test_sample_id, file = errfile)
    else:
        pass
    
    return y_actual_num, y_pred_num

def predict_image_sample_Print(model,X_test, y_test,test_id= -1 ):

    if test_id <0 :
        from random import randrange
        test_sample_id = randrange(len(y))

    else:
        test_sample_id = test_id

    test_image =X_test[test_sample_id]


    #이미지로 만들기
    pixels = np.array(test_image,np.int32)
    plt.imshow(pixels)
    
    test_image = test_image.reshape(1,300,300,3)
    
   
    y_actual = y_test[test_sample_id]
    #print( y_actual)
    y_actual_num = np.argmax(y_actual)
    actual_class = print_class(y_actual_num)
    
    
    y_pred = model.predict(test_image)
    
   
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    predict_class = print_class(y_pred_num)

    #print('실제 클래스 = ', actual_class)
    #print('예측 클래스 = ', predict_class)
    
    
    print('Actual= {}({}) , Predict= {}({})'.format(actual_class,y_actual_num ,predict_class,y_pred_num))
    print("\ny_pred = ", y_pred)
    print('-'*15)
    
    if y_pred_num != y_actual_num:
        with open("틀린 이미지들.txt", "w") as errfile:
            print("%d"%test_sample_id, file=errfile)
    else:
        pass

    
    return y_actual_num, y_pred_num

'''
print(test_data.class_indices,'\n')
model = load_model(' ')
predict_image_sample_Print(model,X, y)
'''

#test 데이터 설정
#path = '  ./images2/test  '
path = input()
test_data = ImageDataGenerator().flow_from_directory(
        path,
        classes = ['food','interior', 'exterior'],
        target_size=(300,300),
        batch_size=100,
        shuffle =True,
        class_mode='categorical')


if __name__ == '__main__' :
    model = load_model(My_Model)
    X_test , y_test = test_data.next()
    corr,count = 0,0
    y_true_mat, y_pred_mat = [] , []
    
    for i in range(len(y)):
        count +=1
        y_act, y_pre = predict_image_sample_Print(model,X_test, y_test)
        y_true_mat.append(y_act)
        y_pred_mat.append(y_pre) 
        if y_act == y_pre :
            corr +=1 
        else:
            continue
    
    print('총횟수 = {} , 맞춘횟수={} , 정확도(Accuracy)={} '.format(count,corr, (corr/count)*100 ) )

from sklearn.metrics import precision_recall_fscore_support as score
import pandas as pd
precision, recall, fscore, support = score(y_true_mat, y_pred_mat)

Confusion_Matrix = pd.DataFrame(np.array([precision, recall, fscore]),
                               columns = ['음식','실내','실외'] ,
                               index  = ["Precision", "Recall", "F1-score"])

print('\n',Confusion_Matrix,'\n')



In [ ]:
image_num = int(input())
test_image =X_test[image_num]

pixels = np.array(test_image,np.int32)
plt.imshow(pixels)

test_image = test_image.reshape(1,300,300,3)
y_actual = y_test[image_num]
y_pred = model.predict(test_image)

y_actual_num = np.argmax(y_actual)
actual_class = print_class(y_actual_num)

y_pred_num = np.argmax(y_pred, axis = 1)[0]
predict_class = print_class(y_pred_num)

print('Actual= {}({}) , Predict= {}({})'.format(actual_class,y_actual_num ,predict_class,y_pred_num))
print("\ny_pred = ", y_pred)
